In [8]:
import pandas as pd
import numpy as np
import cma
from collections import defaultdict

In [ ]:
def fitness_function(weights, skills, hr_requirements, candidates_matrix):
    total_score = 0
    required_levels = np.array([hr_requirements[skill] for skill in skills])
    
    # Apply weights to each skill's importance
    weighted_requirements = required_levels * weights
    
    for candidate_exp in candidates_matrix:
        weighted_experience = candidate_exp * weights
        # Calculate experience gaps
        gaps = np.maximum(0, weighted_requirements - weighted_experience)
        total_score += np.sum(gaps)
    
    return total_score

In [9]:
def analyze_candidate_match(experience, requirements, weights, skills):
    analysis = defaultdict(dict)
    total_gap = 0
    
    for i, skill in enumerate(skills):
        req_exp = requirements[skill]
        actual_exp = experience[i]
        weighted_exp = actual_exp * weights[i]
        gap = max(0, req_exp - actual_exp)
        
        analysis[skill] = {
            'required': req_exp,
            'actual': actual_exp,
            'weighted': weighted_exp,
            'gap': gap,
            'weight': weights[i],
            'meets_requirement': actual_exp >= req_exp
        }
        total_gap += gap * weights[i]
    
    return dict(analysis), total_gap

In [10]:
def rank_candidates(df_talents, hr_requirements):
    
    # Group skills by candidate
    candidate_skills = defaultdict(dict)
    for _, row in df_talents.iterrows():
        candidate_skills[row['talent']][row['skill']] = row['years_experience']
    
    # Prepare data for CMA-ES
    skills_list = list(hr_requirements.keys())
    candidates_data = []
    candidate_names = []
    
    for talent, skills in candidate_skills.items():
        experience = [skills.get(skill, 0) for skill in skills_list]
        if any(exp > 0 for exp in experience):
            candidates_data.append(experience)
            candidate_names.append(talent)
    
    if not candidates_data:
        return []
    
    experience_matrix = np.array(candidates_data)
    
    # Initialize and run CMA-ES
    n_dimensions = len(skills_list)
    initial_weights = np.ones(n_dimensions)
    es = cma.CMAEvolutionStrategy(initial_weights, 0.5, {
        'bounds': [0, 2],
        'maxiter': 100,
        'popsize': 20
    })
    
    while not es.stop():
        solutions = es.ask()
        fitness_values = [
            fitness_function(weights, skills_list, hr_requirements, experience_matrix) 
            for weights in solutions
        ]
        es.tell(solutions, fitness_values)
    
    optimal_weights = es.result.xbest
    
    # Analyze and rank candidates
    ranked_results = []
    for name, experience in zip(candidate_names, candidates_data):
        analysis, total_gap = analyze_candidate_match(
            experience, hr_requirements, optimal_weights, skills_list
        )
        
        # Calculate match percentages
        skill_matches = []
        for skill, details in analysis.items():
            if details['required'] > 0:
                match_percent = min(100, (details['actual'] / details['required']) * 100)
                skill_matches.append((skill, match_percent))
        
        ranked_results.append({
            'name': name,
            'total_gap': total_gap,
            'analysis': analysis,
            'skill_matches': skill_matches
        })
    
    # Sort by total gap (ascending) and create final ranking
    ranked_results.sort(key=lambda x: x['total_gap'])
    
    return ranked_results, optimal_weights

In [11]:
df_talents = pd.read_csv('talent_skills.csv')

hr_requirements = {
    'PHP': 5,        
    'JavaScript': 2,
    'CSS': 3,
    'Node.js': 4,
    'TailwindCSS': 2
}

ranked_candidates, optimal_weights = rank_candidates(df_talents, hr_requirements)

# Display results
print("Talent Ranking Analysis\n")
print("Required Skills and Experience:")
for skill, years in hr_requirements.items():
    print(f"{skill}: {years} years (Weight: {optimal_weights[list(hr_requirements.keys()).index(skill)]:.2f})")

print("\nCandidate Rankings:")
for rank, candidate in enumerate(ranked_candidates, 1):
    print(f"\n{rank}. {candidate['name']}")
    print("Skill Analysis:")
    
    # Sort skills by match percentage for better readability
    sorted_matches = sorted(candidate['skill_matches'], key=lambda x: x[1], reverse=True)
    
    for skill, match_percent in sorted_matches:
        analysis = candidate['analysis'][skill]
        status = "✓" if analysis['meets_requirement'] else "✗"
        print(f"  {status} {skill}:")
        print(f"     Required: {analysis['required']} years")
        print(f"     Actual: {analysis['actual']} years")
        print(f"     Match: {match_percent:.1f}%")
    
    print(f"Overall Gap Score: {candidate['total_gap']:.2f}")
    
    # Add a visual separator between candidates
    if rank < len(ranked_candidates):
        print("-" * 50)

(10_w,20)-aCMA-ES (mu_w=5.9,w_1=27%) in dimension 5 (seed=678564, Tue Oct 22 11:19:12 2024)
Talent Ranking Analysis

Required Skills and Experience:
PHP: 5 years (Weight: 0.00)
JavaScript: 2 years (Weight: 0.00)
CSS: 3 years (Weight: 0.00)
Node.js: 4 years (Weight: 0.00)
TailwindCSS: 2 years (Weight: 0.00)

Candidate Rankings:

1. Kevin Walker
Skill Analysis:
  ✓ PHP:
     Required: 5 years
     Actual: 8 years
     Match: 100.0%
  ✓ JavaScript:
     Required: 2 years
     Actual: 10 years
     Match: 100.0%
  ✓ CSS:
     Required: 3 years
     Actual: 10 years
     Match: 100.0%
  ✓ Node.js:
     Required: 4 years
     Actual: 5 years
     Match: 100.0%
  ✓ TailwindCSS:
     Required: 2 years
     Actual: 4 years
     Match: 100.0%
Overall Gap Score: 0.00
--------------------------------------------------

2. Timothy Hill
Skill Analysis:
  ✓ PHP:
     Required: 5 years
     Actual: 10 years
     Match: 100.0%
  ✓ JavaScript:
     Required: 2 years
     Actual: 10 years
     Match: 100